In [ ]:
# # Segmentaion classes Rescunet_dataset from kaggle (RescueNet 1.0)
# list_of_classes = {'Background':0, 'Debris':1, 'Water':2, 'Building_No_Damage':3, 
#                       'Building_Minor_Damage':4, 'Building_Major_Damage':5, 'Building_Total_Destruction':6, 
#                                  'Vehicle':7, 'Road':8, 'Tree':9, 'Pool':10, 'Sand':11}

In [1]:
import cv2
import numpy as np
import os
import traceback
import shutil

In [4]:
def process_image(image_path, image_paths2, save_dir):
    try:
        # Read and color-convert the labeled image
        image1 = cv2.imread(image_path)
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

        # Define class colors (for visualization)
        class_colors = {
            0: (0, 0, 0),         # Background (Black)
            1: (139, 69, 19),     # Debris (SaddleBrown)
            2: (0, 191, 255),     # Water (DeepSkyBlue)
            3: (50, 205, 50),     # Building_No_Damage (LimeGreen)
            4: (255, 165, 0),     # Building_Minor_Damage (Orange)
            5: (255, 69, 0),      # Building_Major_Damage (RedOrange)
            6: (255, 0, 0),       # Building_Total_Destruction (Red)
            7: (75, 0, 130),      # Vehicle (Indigo)
            8: (169, 169, 169),   # Road (DarkGray)
            9: (34, 139, 34),     # Tree (ForestGreen)
            10: (30, 144, 255),   # Pool (DodgerBlue)
            11: (244, 164, 96)    # Sand (SandyBrown)
        }

        # LUT (Look-Up Table) for mapping the class colors
        lut = np.zeros((256, 1, 3), dtype=np.uint8)
        for class_id, color in class_colors.items():
            lut[class_id] = color
        colored_image = cv2.LUT(image1, lut)

        # Read the original image
        image2 = cv2.imread(image_paths2)

        # Draw polygon annotations
        for class_id, color in class_colors.items():
            if class_id != 0:  # Skip background
                mask = cv2.inRange(colored_image, np.array(color) - 10, np.array(color) + 10)
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                for contour in contours:
                    epsilon = 0.01 * cv2.arcLength(contour, True)
                    approx = cv2.approxPolyDP(contour, epsilon, True)
                    polygon_color = class_colors[class_id]
                    cv2.polylines(image2, [approx], True, polygon_color, 12)  # 12 = thickness of polygon lines
                    # cv2.putText(image2, f"class: {class_id}", (approx[0][0][0], approx[0][0][1]),
                    #             cv2.FONT_HERSHEY_SIMPLEX, 1, polygon_color, 2, cv2.LINE_AA)

        # Saving the image with polygons
        output_filename = os.path.basename(image_paths2)
        output_path = os.path.join(save_dir, os.path.splitext(output_filename)[0] + '_polygon.jpg')
        cv2.imwrite(output_path, image2)

    except Exception as e:
        print(f"Error processing {image_path} and {image_paths2}: {e}")
        traceback.print_exc()

def main():
    image_dir = r"M:\SOL\YOLO\YOLO_RescueNet\Experimantation__Folder\Experiment_cnvrtng_labeled"
    image_dir2 = r"M:\SOL\YOLO\YOLO_RescueNet\Experimantation__Folder\Experiment_cnvrtng_org"
    
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
    image_paths2 = [os.path.join(image_dir2, filename) for filename in os.listdir(image_dir2)]
    
    combined_paths = list(zip(image_paths, image_paths2))
    print(f"Found {len(combined_paths)} image pairs to process.")
    
    # Directory where the output will be saved
    save_dir = r"M:\SOL\YOLO\YOLO_RescueNet\Experimantation__Folder\Cnvrtd_train_image"
    
    # Clear the directory only once, before processing all images
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    else:
        for filename in os.listdir(save_dir):
            file_path = os.path.join(save_dir, filename)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')
    
    # Process all image pairs
    for image_path, image_paths2 in combined_paths:
        process_image(image_path, image_paths2, save_dir)

if __name__ == '__main__':
    main()

Found 82 image pairs to process.
